# Script for files WITH pre-defined phase values


1. Import Libraries


In [5]:
from google.colab import drive
import numpy as np

2. Connect to GDrive if needed

In [6]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


3. Helper Function 1 - <br>
To create structured data from the file

In [7]:
# Helper Function - create structured data from the file

def get_data(j,pair_paths,data,max_pairs):
    result = []
    for i in range(pair_paths):
        line = data[j].decode('utf-8').strip().split()

        Tx = data[j+2].decode('utf-8').strip().split()
        Tx = [float(i) for i in Tx]
        j = j+int(line[1]) + 4
        power = float(line[2])
        aoa = float(line[3])
        diff = int(line[1])
        Rx = data[j-1].decode('utf-8').strip().split()

        Rx = [float(i) for i in Rx]
        tmp = [Tx,Rx,[power,aoa,diff]]
        result.append(tmp)
    # print(Tx, Rx, [power, aoa, diff])
    result = np.array(result)
    return j,result

4. Helper Function 2 - <br>
To get Power and Delay lists from the structured data

In [8]:
# Helper Function - get Power, Delay and Phase lists from the structured data

def getPowerDelayPhase(rawData):
        power = []
        delay = []
        phase = []
        for i in range(len(rawData)):
          tmp = []
          d = []
          p = []
          if rawData[i].size != 0:                             #change identation of power.append and change pos of tmp = [] to make len less than 29929
            for j in range(len(rawData[i])):
              tmp.append(rawData[i][j][0])
              d.append(rawData[i][j][1])
              p.append(rawData[i][j][2])
          power.append(np.array(tmp))
          delay.append(np.array(d))
          phase.append(np.array(p))
        return power,delay,phase

5. Helper Function 3 - <br>
To get all Rx and Tx values from the structured data

In [9]:
# Helper Function - get all Rx and Tx values from the structured data

def getRxTx(rawData):
        Rx_position = []
        Tx_position = []
        for i in range(len(rawData)):
          tmp = []
          d = []
          if rawData[i].size != 0:
            for j in range(len(rawData[i])):
              tmp.append(rawData[i][j][0])
              d.append(rawData[i][j][1])
          Rx_position.append(np.array(d))
          Tx_position.append(np.array(tmp))
        return Tx_position,Rx_position

6. Helper Function 4 - <br>
To remove data with empty values

In [10]:
# Helper Function - remove data with 0 values

def removeEmpty(l):
  filtered_list = [arr for arr in l if arr.size > 0]
  return filtered_list

7. Helper Function 5 - <br>
To check if the Rx and Tx values are the same throughout the data

In [11]:
# Helper Function - check if the Rx and Tx values are the same throughout the data

def checkArray(l):
  indices = {}
  for i in range(len(l)):
   comparison_results = [np.array_equal(arr, l[i][0]) for arr in l[i]]
   for j in range(len(comparison_results)):
    if comparison_results[j] == False:
      indices[i] = j
  return indices

8. Helper Function 6 - <br>
To convert the extracted phase values from degrees to radians

In [32]:
# Helper Function - Convert phase from degrees to radians

def convertPhase(phase):
  # print(phase.shape, type(phase))
  tmp = (np.pi /180)
  res = []
  for p in phase:
    res.append(p*tmp)
  return res

9. Helper Function 7 - <br>
To get structured data from the file containing power, delay and phase values

In [21]:
# Helper Function - To get structured data from the file containing power, delay and phase values
def getPowerdata(j,pair_paths,data):
    result = []
    for i in range(pair_paths):
        line = data[j].decode('utf-8').strip().split()

        power = float(line[3])
        phase = float(line[1])
        delay = float(line[2])
        tmp = [power, delay, phase]
        result.append(tmp)
        # print(j)
        j+=1
    # print(Tx, Rx, [power, aoa, diff])
    result = np.array(result)
    return j,result

# Main Function 1 <br>
Get structured data from the raw p2m file


In [13]:
# Main Function 1
def getData(file):
    final = []
    with open(file, 'rb') as file:
      data = file.readlines()
      total_pairs = int(data[1].decode('utf-8').strip())
      pair_paths = int(data[2].decode('utf-8').strip().split()[1])
      # print(total_pairs, pair_paths)
      j = 4
      j,result = get_data(j,pair_paths,data,pair_paths)
      final.append(result)

      for i in range(total_pairs - 1):
        pair_paths = int(data[j].decode('utf-8').strip().split()[1])

        if pair_paths == 0:
          j,result = get_data(j + 1,pair_paths,data,pair_paths)
          final.append(result)
        else:
          j,result = get_data(j + 2,pair_paths,data,pair_paths)
          final.append(result)
    Tx_position = []
    Rx_position = []
    Tx_position, Rx_position = getRxTx(final)
    # power, delay = getPowerDelay(final)
    return Tx_position, Rx_position

# Main Function 2 <br>
To extract the Power, Phase and Delay values

In [25]:
# Main Function 2
def getPowerandDelay(cirFilename):
    final = []
    with open(cirFilename, 'rb') as file:
      data = file.readlines()
      total_pairs = int(data[3].decode('utf-8').strip())
      pair_paths = int(data[4].decode('utf-8').strip().split()[1])
      j = 5
      j, result = getPowerdata(j,pair_paths,data)
      final.append(result)
      for i in range(total_pairs - 1):
        pair_paths = int(data[j].decode('utf-8').strip().split()[1])
        # print(pair_paths)
        if pair_paths == 0:
          j,result = getPowerdata(j + 1,pair_paths,data)
          final.append(result)
        else:
          j,result = getPowerdata(j + 1,pair_paths,data)
          final.append(result)
    power,delay,phase = getPowerDelayPhase(final)
    phase = convertPhase(phase)
    return power, delay, phase

# Main Function 3 <br>
To get data without repetition and empty arrays

In [35]:
 # Main Function 3
def getDataWitoutEmptyRepeat(Tx_position, Rx_position, power, delay, phase):
  Tx_position = removeEmpty(Tx_position)
  Rx_position = removeEmpty(Rx_position)
  power = removeEmpty(power)
  delay= removeEmpty(delay)
  phase = removeEmpty(phase)
  indices = checkArray(Tx_position)

  if indices == {}:
    Tx = Tx_position[0][0]

  indices = checkArray(Rx_position)

  if indices == {}:
    Rx = []
    for i in range(len(Rx_position)):
      Rx.append(Rx_position[i][0])

  return Tx, Rx, power, delay,phase

# Main Function 4 <br>
To calculate the channel frequency using the predefined phase values

In [43]:
# Main Function 4
def getDataChannelFrequency(power, delay, theta, M, bW):
  channelFreq = []
  for i in range(len(power)):
    newPower = 10 ** ((power[i] - 30) / 20)
    comp = np.zeros((len(power[i]), ), dtype=np.complex128)
    comp = np.exp(1j * theta[i])
    newPower = newPower * comp

    freq = np.linspace(-bW/2, bW/2 - bW/M, M)
    tmp = []
    for j in range(len(freq)):
      h = np.exp(1j * 2 * np.pi * freq[j] * delay[i])
      a = newPower * h
      tmp.append(sum(a))


    channelFreq.append(tmp)
  channelFreq = np.array(channelFreq)
  return channelFreq

# Main Function 5 <br>
TO calculate channel frequency without removing the empty arrays

In [45]:
# Main Function 5
# Channel Frequency Version 3
def getDataChannelFreqWithEmpty(Tx_position, Rx_position, power, delay, phase, M, bW):
    indices = checkArray(Tx_position)

    if indices == {}:
      Tx = Tx_position[0][0]

    indices = checkArray(Rx_position)

    if indices == {}:
      Rx = []
      for i in range(len(Rx_position)):
        if Rx_position[i].size != 0:
          Rx.append(Rx_position[i][0])
        else:
          Rx.append([])
    channelFreq = []
    for i in range(len(power)):
      theta = phase[i]
      newPower = 10 ** ((power[i] - 30) / 20)
      comp = np.zeros((len(power[i]), ), dtype=np.complex128)
      comp = np.exp(1j * theta)
      newPower = newPower * comp

      freq = np.linspace(-bW/2, bW/2 - bW/M, M)
      tmp = []
      for j in range(len(freq)):
        h = np.exp(1j * 2 * np.pi * freq[j] * delay[i])
        a = newPower * h
        tmp.append(sum(a))


      channelFreq.append(tmp)
    channelFreq = np.array(channelFreq)
    return channelFreq

In [16]:
fileName = '' # Include path of the paths file

In [49]:
cirFilename = '' # Include path of the 'cir' file

In [18]:
Tx_position, Rx_position = getData(fileName)

In [33]:
power, delay, phase = getPowerandDelay(cirFilename)

In [37]:
Tx, Rx, power, delay, phase = getDataWitoutEmptyRepeat(Tx_position, Rx_position, power, delay,phase)

We have 4 lists for Tx, Rx, and Power and Delay values.
The variable names and dimension for each are-

*   Tx - 1 x 3
*   Rx - 27099 x 3
*   Power - 27099 x (varying lengths)
*   Delay - 27099 x (varying lengths)




In [46]:
M = 200
bW = 400

In [47]:
channelFreq = getDataChannelFrequency(power, delay,phase, M, bW)

In [50]:
channelFreq = getDataChannelFreqWithEmpty(Tx_position, Rx_position, power, delay, phase, M, bW)